<a href="https://colab.research.google.com/github/aadityasanjay0801/llm-fine-tuning/blob/main/LLM_Finetuning_without_Blurr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine Tuning a LLM for News Summarization using Hugging Face**

## **Loading Dependencies**

In [1]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


In [2]:
!pip install datasets -q

In [3]:
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00


In [4]:
!pip install accelerate -q

In [5]:
!pip install torchmetrics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 4.4 MB/s eta 0:00:00


In [6]:
!pip install bert-score -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 816.2 kB/s eta 0:00:00


In [7]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [57]:
from bert_score import BERTScorer
from bert_score import score
from evaluate import load
import pprint
from torchmetrics.text.bert import BERTScore


In [26]:
from tqdm.notebook import tqdm

## **Importing Pretrained Model and its Tokenizer**

In [9]:
model_name='google/flan-t5-small'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# **Preparing Dataset**

In [10]:
# prompt: load news_summaries.csv in a df

df = pd.read_csv('news_summaries (1).csv')
df = df.reset_index(drop=True)



In [11]:
df = df[['news', 'summary']]

In [12]:
train_ratio = 0.7

# Randomly shuffle the dataframe
df = df.sample(frac=1)

# Calculate the number of rows for train and test datasets
train_size = int(train_ratio * len(df))
test_size = len(df) - train_size

# Split the dataframe into train and test datasets
train_data = df[:train_size]
test_data = df[train_size:]

## **Creating Huggingface dataset**

In [13]:
huggingface_df_train = Dataset.from_pandas(train_data)
huggingface_df_test = Dataset.from_pandas(test_data)

huggingface_df = DatasetDict()

huggingface_df['train'] = huggingface_df_train
huggingface_df['test'] = huggingface_df_test


## **Tokenizing Data**

In [14]:
def tokenize_function(example):
    start_prompt = 'Summarize the following news.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + news + end_prompt for news in example["news"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids.to('cuda')
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids.to('cuda')

    return example

In [15]:
# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = huggingface_df.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['news', 'summary'])

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

## **Get Number of Trainable Parameters in Model**

In [16]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


## **Try Inference from Pre trained Model**

In [17]:
index = 200

news = df['news'][4]
summary = df['summary'][4]

prompt = f"""
Summarize the following news.

{news}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=600,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following news.

Worldcom director ends evidence

The former chief financial officer at US telecoms firm WorldCom has finished giving evidence at the trial of his ex-boss Bernie Ebbers.

Scott Sullivan admitted to jurors he was willing to commit fraud to meet Wall Street earnings projections. Mr Ebbers is on trial for fraud and conspiracy in relation to WorldCom's collapse in 2002. He pleads not guilty. Mr Sullivan has spent two days being cross-examined by lawyers for former Worldcom chief executive Mr Ebbers.

Attorney Reid Weingarten has attempted to portray Mr Sullivan as a liar and on Thursday quizzed him about his decision to commit fraud to meet analysts' profit estimates. "At that point in time," Mr Sullivan said, referring to the first false entries in late 2000, "I knew it was wrong and I knew it was against the law, but I thought we would get throu

## **Tryin LoRA to Finetune Model**

In [18]:
from peft import LoraConfig, get_peft_model, TaskType

## **Defining LoRA Config**

In [19]:
lora_config = LoraConfig(
    r=64, # Rank
    lora_alpha=32,
    target_modules=["k", "q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

## **Adding LoRA Adapters to Model**

In [20]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 4128768
all model parameters: 81089920
percentage of trainable model parameters: 5.09%


## **Defining Trainer and start training**

In [21]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=500,
    logging_steps=10,
    max_steps=500
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset= tokenized_datasets['train'],
)

In [22]:
peft_trainer.train()

peft_model_path="./peft-news-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,24.437500
20,11.856200
30,5.678100
40,3.732800
50,3.293800
60,3.187500
70,3.087500
80,3.034400
90,2.884400
100,2.703100


('./peft-news-summary-checkpoint-local/tokenizer_config.json',
 './peft-news-summary-checkpoint-local/special_tokens_map.json',
 './peft-news-summary-checkpoint-local/tokenizer.json')

In [71]:
peft_trainer.save_model('manual-saving')

In [23]:
original_model.to('cuda')


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=384, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=512, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=384, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): lora.Linear(
                (base_layer): Linear(in_features=512, o

In [24]:
# instruct_model = AutoModelForSeq2SeqLM.from_pretrained("/content/peft-dialogue-summary-checkpoint-local", torch_dtype=torch.bfloat16, repo_type = 'local')
# instruct_model = instruct_model.to('cuda')


## **Getting Inference from tuned model**

In [25]:
index = 200

news = df['news'][index]
summary = df['summary'][index]

prompt = f"""
Summarize the following news.

{news}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=1000,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - After Finetuing:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following news.

French consumer spending rising

French consumers increased their spending by 1.5% in January, a figure which bodes well for the country's economic growth, figures revealed.

The National Statistic Institute (INSEE) added that consumer spending in January rose 3.8% on a year-on-year basis. Rising sales of household equipment were behind the increase. The INSEE also said that French consumer prices fell 0.6% in January, but were up 1.6% on an annual basis.

Despite the general increase in spending in January, French households bought fewer cars in January. According to the INSEE, car sales fell 2.8% in January, following a fall of 0.6% in December. But on a year-on-year basis, the sector still saw a sales increase of 6.5%. Consumer spending fuelled France's economic growth in the last quarter of 2004 and analysts expect that it will continue t

In [ ]:
print(f"Length of human summary: {len(summary)}")
print(f"Length of Model geenrated summary: {len(output)}")


Length of human summary: 554
Length of Model geenrated summary: 995


# **Evaluating Model**

In [36]:
# test_data = huggingface_df['test']
sample = huggingface_df['test'].select(range(20))
references = []
predictions = []
for example in tqdm(sample):
  references.append(example['summary'])
# references = [example['summary'] for example in huggingface_df['test']]
  news = example['news']
  summary = example['summary']

  prompt = f"""
  Summarize the following news.

  {news}

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
  output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=1000,
    )[0],
    skip_special_tokens=True
)
  predictions.append(output)




  0%|          | 0/20 [00:00<?, ?it/s]

## **Calculating BERT Score**

In [60]:
bertscore = load("bertscore")
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'precision': [0.9249758720397949, 0.84720379114151, 0.9496321678161621, 0.8823515176773071, 0.8522284030914307, 0.870952844619751, 0.7926883697509766, 0.7817142009735107, 0.8474092483520508, 0.859169602394104, 0.9109368324279785, 0.7158756256103516, 0.9323558807373047, 0.86892169713974, 0.7629482746124268, 0.935592770576477, 0.8742504119873047, 0.8950560092926025, 0.8042558431625366, 0.8520621657371521], 'recall': [0.7260661125183105, 0.7374225854873657, 0.7373521327972412, 0.7091840505599976, 0.7545222043991089, 0.8317270874977112, 0.8107845187187195, 0.7726324200630188, 0.834888219833374, 0.6883458495140076, 0.7395406365394592, 0.6360564827919006, 0.7702025771141052, 0.9145275950431824, 0.7902119755744934, 0.8116356134414673, 0.8211069107055664, 0.8334848284721375, 0.8154115080833435, 0.7649902701377869], 'f1': [0.8135390877723694, 0.7885103821754456, 0.8301361203193665, 0.7863470315933228, 0.8004046082496643, 0.850888192653656, 0.8016343116760254, 0.7771468162536621, 0.841102123260

In [68]:
precision = sum(results['precision'])/len(results['precision'])
recall = sum(results['recall'])/len(results['recall'])
f1 = sum(results['f1'])/len(results['f1'])
results_average = {}
results_average['precision'] = precision
results_average['recall'] = recall
results_average['f1'] = f1



In [69]:
pprint.pprint(results_average)

{'f1': 0.8125120490789414,
 'precision': 0.8580290764570236,
 'recall': 0.7750046789646149}


20